In [ ]:
import os
import caer 
import canaro
import numpy as np
import cv2 as cv
import gc


In [ ]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'data\simpsons_dataset'


In [ ]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))

# sort in descending order
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

In [ ]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break
characters

In [ ]:
# create training data
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)


In [ ]:
len(train)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap = 'gray')
plt.show()


In [ ]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical
# normalize the featureSet ==> (0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))


In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)

In [ ]:
del train
del featureSet
del labels
gc.collect()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
# image data generator
datagen = canaro.generators.imageDataGenerator() # very simple image generator using keras library
train_gen = datagen.flow(x_train, y_train, batch_size = BATCH_SIZE)


In [ ]:
# creating the model
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters),
loss='binary_crossentropy', decay=1e-6, learning_rate=0.001,
momentum=0.09, nesterov=True)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
training = model.fit(train_gen, steps_per_epoch=len(x_train)//BATCH_SIZE, epochs=EPOCHS, validation_data=(x_val, y_val), validation_steps=len(y_val)//BATCH_SIZE,callbacks = callbacks_list)

In [ ]:
characters

In [ ]:
test_path = r'C:\Users\MCBRNCK1\Documents\GitHub\opencv_BasicsWithExamples\data\kaggle_simpson_testset\kaggle_simpson_testset\bart_simpson_1.jpg'
img = cv.imread(test_path)

plt.imshow(img)
plt.show()


In [ ]:
def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [ ]:
predictions = model.predict(prepare(img))


In [ ]:
print(charachters[np.argmax(predictions[0])])